# 회원탈퇴예측 모델링

## 1 .데이터 불러오기

In [1]:
import pandas as pd
pd.set_option('mode.chained_assignment',  None) # <==== 경고를 끈다

In [2]:
customer = pd.read_csv('customer_join.csv')
use_log_month = pd.read_csv('use_log_month.csv')

In [3]:
customer

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,0_종일,10500,2_일반,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,0_종일,10500,2_일반,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,0_종일,10500,2_일반,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,0_종일,10500,2_일반,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,2_야간,6000,2_일반,3.916667,4.0,6,1,1,2019-04-30,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4187,HD676663,XXXX,C01,M,2019-03-14,NaN,CA1,0,0_종일,10500,2_일반,8.000000,8.0,8,8,0,2019-04-30,1
4188,HD246549,XXXXX,C01,F,2019-03-14,NaN,CA1,0,0_종일,10500,2_일반,10.000000,10.0,10,10,0,2019-04-30,1
4189,GD037007,XXXXX,C03,M,2019-03-14,NaN,CA1,0,2_야간,6000,2_일반,8.000000,8.0,8,8,0,2019-04-30,1
4190,OA953150,XXXXX,C01,M,2019-03-14,NaN,CA1,0,0_종일,10500,2_일반,11.000000,11.0,11,11,0,2019-04-30,1


In [4]:
use_log_month

,연월,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7
...,...,...,...
36837,201903,TS995853,8
36838,201903,TS998593,8
36839,201903,TS999079,3
36840,201903,TS999231,6


In [5]:
#현재 월 기준으로 한달 전 이용 횟수 계산
year_month = list(use_log_month['연월'].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_month)):
    
    tmp = use_log_month.loc[use_log_month['연월'] == year_month[i]]
    tmp.rename(columns = {'count' : 'count_0'}, inplace=True)
    tmp_before = use_log_month.loc[use_log_month['연월'] == year_month[i-1]]
    
    del tmp_before['연월']
    
    tmp_before.rename(columns = {'count' : 'count_1'}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on='customer_id', how='left')
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog

,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0
...,...,...,...,...
33846,201903,TS995853,8,11.0
33847,201903,TS998593,8,7.0
33848,201903,TS999079,3,2.0
33849,201903,TS999231,6,6.0


In [6]:
from dateutil.relativedelta import relativedelta

## 2. 탈퇴 회원 데이터 프레임 만들기

In [7]:
# 탈퇴한 인원 조회
exit_customer = customer.loc[customer['is_deleted'] == 1]
exit_customer['exit_date'] = None
exit_customer

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,0_종일,10500,2_일반,3.0,3.0,3,3,0,2018-04-30,23,None
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,1_주간,7500,2_일반,3.0,3.0,3,3,0,2018-04-30,23,None
730,HD542886,XX,C01,M,2016-05-01,2018-04-30,CA1,1,0_종일,10500,2_일반,1.0,1.0,1,1,0,2018-04-30,23,None
770,HD597545,XXXXX,C03,F,2016-06-01,2018-05-31,CA1,1,2_야간,6000,2_일반,3.5,3.5,4,3,1,2018-05-31,23,None
785,HI749296,XXXXX,C01,M,2016-06-01,2018-05-31,CA1,1,0_종일,10500,2_일반,3.0,3.0,3,3,0,2018-05-31,23,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4049,OA143589,XXXXX,C03,F,2019-01-15,2019-02-28,CA1,1,2_야간,6000,2_일반,4.5,4.5,6,3,0,2019-02-28,1,None
4050,GD367382,XXXX,C01,M,2019-01-15,2019-03-31,CA1,1,0_종일,10500,2_일반,7.0,6.0,12,3,0,2019-03-31,2,None
4068,OA034442,XXXXX,C02,M,2019-02-05,2019-03-31,CA1,1,1_주간,7500,2_일반,7.0,7.0,11,3,0,2019-03-31,1,None
4086,PL508278,XXXXX,C03,M,2019-02-09,2019-03-31,CA1,1,2_야간,6000,2_일반,7.0,7.0,10,4,0,2019-03-31,1,None


In [8]:
exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])
#탈퇴한 인원의 탈퇴 신청은 1달 전에 신청하기 때문에 end_date 보다 1달 전 데이터를 탈퇴 데이터로 처리
for i in range(len(exit_customer)):

    exit_customer['exit_date'].iloc[i] = exit_customer['end_date'].iloc[i] - relativedelta(months=1)

exit_customer['exit_date'] = pd.to_datetime(exit_customer['exit_date'], format='%Y%m')
exit_customer['연월'] = exit_customer['exit_date'].dt.strftime('%Y%m')
uselog['연월'] = uselog['연월'].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=['customer_id', '연월'], how='left')
print(len(uselog))
exit_uselog.head()

33851


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [9]:
# 고객 이름 기준으로 null 행 제거
exit_uselog = exit_uselog.dropna(subset=['name'])
print(len(exit_uselog))
print(len(exit_uselog['customer_id'].unique()))
exit_uselog.head()

1104
1104


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,2_일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,2_일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,2_일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,2_일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,2_일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30


## 3. 지속회원 데이터 프레임 만들기

In [10]:
uselog

,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0
...,...,...,...,...
33846,201903,TS995853,8,11.0
33847,201903,TS998593,8,7.0
33848,201903,TS999079,3,2.0
33849,201903,TS999231,6,6.0


In [11]:
stay_customer = customer.loc[customer['is_deleted'] == 0]
stay_uselog = pd.merge(uselog, stay_customer, on='customer_id', how='left')
print(len(stay_uselog))

33851


In [12]:
stay_uselog = stay_uselog.dropna(subset=['name'])
stay_uselog

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201805,AS002855,5,4.0,XXXX,C03,F,2016-11-01,NaN,CA1,...,2_야간,6000.0,2_일반,4.500000,5.0,7.0,2.0,1.0,2019-04-30,29.0
1,201805,AS009373,4,3.0,XX,C01,F,2015-11-01,NaN,CA1,...,0_종일,10500.0,2_일반,5.083333,5.0,7.0,3.0,1.0,2019-04-30,41.0
2,201805,AS015233,7,NaN,XXXXX,C01,M,2018-05-13,NaN,CA2,...,0_종일,10500.0,0_입회비반액할인,7.545455,7.0,11.0,4.0,1.0,2019-04-30,11.0
3,201805,AS015315,3,6.0,XXXXX,C01,M,2015-07-01,NaN,CA1,...,0_종일,10500.0,2_일반,4.833333,5.0,7.0,3.0,1.0,2019-04-30,45.0
4,201805,AS015739,5,7.0,XXXXX,C03,M,2017-06-01,NaN,CA1,...,2_야간,6000.0,2_일반,5.583333,5.5,8.0,4.0,1.0,2019-04-30,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33846,201903,TS995853,8,11.0,XXXX,C01,M,2019-02-08,NaN,CA1,...,0_종일,10500.0,2_일반,9.500000,9.5,11.0,8.0,1.0,2019-04-30,2.0
33847,201903,TS998593,8,7.0,XXXXX,C03,M,2018-09-01,NaN,CA1,...,2_야간,6000.0,2_일반,8.142857,8.0,9.0,7.0,1.0,2019-04-30,7.0
33848,201903,TS999079,3,2.0,XXX,C03,M,2016-06-01,NaN,CA1,...,2_야간,6000.0,2_일반,4.916667,5.5,9.0,2.0,1.0,2019-04-30,34.0
33849,201903,TS999231,6,6.0,XXXX,C01,M,2017-03-01,NaN,CA1,...,0_종일,10500.0,2_일반,4.666667,5.0,8.0,1.0,1.0,2019-04-30,25.0


In [13]:
stay_uselog = stay_uselog.sample(frac=1).reset_index(drop=True)  # 데이터를 섞음
stay_uselog = stay_uselog.drop_duplicates(subset='customer_id')   # 고객 ID 중복 행 제거
print(len(stay_uselog))
stay_uselog

2842


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201808,OA796761,10,NaN,XX,C02,F,2018-08-03,NaN,CA1,...,1_주간,7500.0,2_일반,8.875000,9.0,10.0,7.0,1.0,2019-04-30,8.0
1,201811,IK758927,8,8.0,XXXXXX,C01,M,2018-06-03,NaN,CA2,...,0_종일,10500.0,0_입회비반액할인,7.000000,7.0,9.0,5.0,1.0,2019-04-30,10.0
2,201805,HI606562,8,7.0,XXX,C01,F,2017-04-01,NaN,CA1,...,0_종일,10500.0,2_일반,5.166667,5.5,8.0,2.0,1.0,2019-04-30,24.0
3,201806,PL093071,2,6.0,XXXX,C01,F,2016-01-01,NaN,CA1,...,0_종일,10500.0,2_일반,4.916667,5.5,8.0,2.0,1.0,2019-04-30,39.0
4,201903,PL186773,6,5.0,XXXXX,C03,M,2016-02-01,NaN,CA1,...,2_야간,6000.0,2_일반,4.000000,4.5,7.0,2.0,1.0,2019-04-30,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26229,201903,OA953150,11,NaN,XXXXX,C01,M,2019-03-14,NaN,CA1,...,0_종일,10500.0,2_일반,11.000000,11.0,11.0,11.0,0.0,2019-04-30,1.0
26257,201903,TS028201,9,NaN,XXXXX,C01,M,2019-03-05,NaN,CA1,...,0_종일,10500.0,2_일반,9.000000,9.0,9.0,9.0,1.0,2019-04-30,1.0
26750,201903,OA643695,11,NaN,XXXXX,C02,F,2019-03-12,NaN,CA1,...,1_주간,7500.0,2_일반,11.000000,11.0,11.0,11.0,0.0,2019-04-30,1.0
27097,201903,IK439676,10,NaN,XXXX,C03,M,2019-03-01,NaN,CA1,...,2_야간,6000.0,2_일반,10.000000,10.0,10.0,10.0,1.0,2019-04-30,1.0


In [14]:
customer_df = pd.concat([stay_uselog, exit_uselog], ignore_index=True)
customer_df

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201808,OA796761,10,NaN,XX,C02,F,2018-08-03,NaN,CA1,...,7500.0,2_일반,8.875000,9.0,10.0,7.0,1.0,2019-04-30,8.0,NaT
1,201811,IK758927,8,8.0,XXXXXX,C01,M,2018-06-03,NaN,CA2,...,10500.0,0_입회비반액할인,7.000000,7.0,9.0,5.0,1.0,2019-04-30,10.0,NaT
2,201805,HI606562,8,7.0,XXX,C01,F,2017-04-01,NaN,CA1,...,10500.0,2_일반,5.166667,5.5,8.0,2.0,1.0,2019-04-30,24.0,NaT
3,201806,PL093071,2,6.0,XXXX,C01,F,2016-01-01,NaN,CA1,...,10500.0,2_일반,4.916667,5.5,8.0,2.0,1.0,2019-04-30,39.0,NaT
4,201903,PL186773,6,5.0,XXXXX,C03,M,2016-02-01,NaN,CA1,...,6000.0,2_일반,4.000000,4.5,7.0,2.0,1.0,2019-04-30,38.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3941,201902,TS645212,4,2.0,XXXX,C03,F,2018-03-01,2019-03-31 00:00:00,CA1,...,6000.0,2_일반,4.500000,4.5,7.0,1.0,0.0,2019-03-31,12.0,2019-02-28
3942,201902,TS741703,5,6.0,XXXX,C03,M,2018-12-08,2019-03-31 00:00:00,CA3,...,6000.0,1_입회비무료,6.250000,6.0,8.0,5.0,0.0,2019-03-31,3.0,2019-02-28
3943,201902,TS859258,1,3.0,XXXXX,C02,F,2018-12-07,2019-03-31 00:00:00,CA3,...,7500.0,1_입회비무료,2.500000,2.0,5.0,1.0,0.0,2019-03-31,3.0,2019-02-28
3944,201902,TS886985,5,3.0,XXX,C02,F,2018-03-01,2019-03-31 00:00:00,CA1,...,7500.0,2_일반,4.250000,4.0,7.0,2.0,1.0,2019-03-31,12.0,2019-02-28


In [15]:
customer_df.to_csv('customer_final.csv', index=False)

## 4. 예측할 달의 재적 기간

In [16]:
customer_df['period'] = 0
customer_df['now_date'] = pd.to_datetime(customer_df['연월'], format='%Y%m')
customer_df['start_date'] = pd.to_datetime(customer_df['start_date'])

In [17]:
for i in range(len(customer_df)):
    delta = relativedelta(customer_df['now_date'][i], customer_df['start_date'][i])
    customer_df['period'][i]=int(delta.years*12 + delta.months)

customer_df

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201808,OA796761,10,NaN,XX,C02,F,2018-08-03,NaN,CA1,...,8.875000,9.0,10.0,7.0,1.0,2019-04-30,8.0,NaT,0,2018-08-01
1,201811,IK758927,8,8.0,XXXXXX,C01,M,2018-06-03,NaN,CA2,...,7.000000,7.0,9.0,5.0,1.0,2019-04-30,10.0,NaT,4,2018-11-01
2,201805,HI606562,8,7.0,XXX,C01,F,2017-04-01,NaN,CA1,...,5.166667,5.5,8.0,2.0,1.0,2019-04-30,24.0,NaT,13,2018-05-01
3,201806,PL093071,2,6.0,XXXX,C01,F,2016-01-01,NaN,CA1,...,4.916667,5.5,8.0,2.0,1.0,2019-04-30,39.0,NaT,29,2018-06-01
4,201903,PL186773,6,5.0,XXXXX,C03,M,2016-02-01,NaN,CA1,...,4.000000,4.5,7.0,2.0,1.0,2019-04-30,38.0,NaT,37,2019-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3941,201902,TS645212,4,2.0,XXXX,C03,F,2018-03-01,2019-03-31 00:00:00,CA1,...,4.500000,4.5,7.0,1.0,0.0,2019-03-31,12.0,2019-02-28,11,2019-02-01
3942,201902,TS741703,5,6.0,XXXX,C03,M,2018-12-08,2019-03-31 00:00:00,CA3,...,6.250000,6.0,8.0,5.0,0.0,2019-03-31,3.0,2019-02-28,1,2019-02-01
3943,201902,TS859258,1,3.0,XXXXX,C02,F,2018-12-07,2019-03-31 00:00:00,CA3,...,2.500000,2.0,5.0,1.0,0.0,2019-03-31,3.0,2019-02-28,1,2019-02-01
3944,201902,TS886985,5,3.0,XXX,C02,F,2018-03-01,2019-03-31 00:00:00,CA1,...,4.250000,4.0,7.0,2.0,1.0,2019-03-31,12.0,2019-02-28,11,2019-02-01


## 5. 데이터가공

- 결측치 처리

In [18]:
customer_df.isnull().sum()

연월                      0
customer_id             0
count_0                 0
count_1               257
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [19]:
# end_date와 exit_date는 탈퇴한 회원만 있고 지속 회원은 null값이므로 count_1의 결측치 행만 제거
customer_df = customer_df.dropna(subset=['count_1'])
customer_df.isnull().sum()

연월                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2637
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2637
period                  0
now_date                0
dtype: int64

- 문자열 변수 처리

In [20]:
# 설명변수 추출
feature_col = ['campaign_name','class_name','gender','count_1','routine_flg','period','is_deleted']
customer_df = customer_df[feature_col]
customer_df

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
1,0_입회비반액할인,0_종일,M,8.0,1.0,4,0.0
2,2_일반,0_종일,F,7.0,1.0,13,0.0
3,2_일반,0_종일,F,6.0,1.0,29,0.0
4,2_일반,2_야간,M,5.0,1.0,37,0.0
5,2_일반,0_종일,M,4.0,1.0,28,0.0
...,...,...,...,...,...,...,...
3941,2_일반,2_야간,F,2.0,0.0,11,1.0
3942,1_입회비무료,2_야간,M,6.0,0.0,1,1.0
3943,1_입회비무료,1_주간,F,3.0,0.0,1,1.0
3944,2_일반,1_주간,F,3.0,1.0,11,1.0


In [21]:
customer_df = pd.get_dummies(customer_df)
customer_df

,count_1,routine_flg,period,is_deleted,campaign_name_0_입회비반액할인,campaign_name_1_입회비무료,campaign_name_2_일반,class_name_0_종일,class_name_1_주간,class_name_2_야간,gender_F,gender_M
1,8.0,1.0,4,0.0,1,0,0,1,0,0,0,1
2,7.0,1.0,13,0.0,0,0,1,1,0,0,1,0
3,6.0,1.0,29,0.0,0,0,1,1,0,0,1,0
4,5.0,1.0,37,0.0,0,0,1,0,0,1,0,1
5,4.0,1.0,28,0.0,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3941,2.0,0.0,11,1.0,0,0,1,0,0,1,1,0
3942,6.0,0.0,1,1.0,0,1,0,0,0,1,0,1
3943,3.0,0.0,1,1.0,0,1,0,0,1,0,1,0
3944,3.0,1.0,11,1.0,0,0,1,0,1,0,1,0


In [22]:
# 컬럼이 많기 때문에 각 문자열 변수 더미 컬럼을 하나씩 삭제
customer_df = customer_df.drop('campaign_name_2_일반', axis=1)
customer_df = customer_df.drop('class_name_2_야간', axis=1)
customer_df = customer_df.drop('gender_M', axis=1)

In [23]:
customer_df

,count_1,routine_flg,period,is_deleted,campaign_name_0_입회비반액할인,campaign_name_1_입회비무료,class_name_0_종일,class_name_1_주간,gender_F
1,8.0,1.0,4,0.0,1,0,1,0,0
2,7.0,1.0,13,0.0,0,0,1,0,1
3,6.0,1.0,29,0.0,0,0,1,0,1
4,5.0,1.0,37,0.0,0,0,0,0,0
5,4.0,1.0,28,0.0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
3941,2.0,0.0,11,1.0,0,0,0,0,1
3942,6.0,0.0,1,1.0,0,1,0,0,0
3943,3.0,0.0,1,1.0,0,1,0,1,1
3944,3.0,1.0,11,1.0,0,0,0,1,1


## 6. 의사결정 트리 모델링

In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [25]:
x = customer_df.drop('is_deleted', axis=1)
x = x.sample(len(x))  # x 데이터 섞기
y = customer_df['is_deleted']
y = y.loc[x.index] # y 데이터도 x 데이터에 맞도록 순서 배치

In [26]:
x_train, x_test, y_train, y_test = train_test_split(x,y)

In [27]:
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print('train data set 정확도:' ,model.score(x_train,y_train))
print('test data set 정확도',model.score(x_test,y_test))
print('test data set 정확도', accuracy_score(y_test, y_pred))

train data set 정확도: 0.9779464931308749
test data set 정확도 0.9230769230769231
test data set 정확도 0.9230769230769231


 - 모델 튜닝 : max_depth를 추가하여 테스트 데이터 정확도가 94%까지 성능 향상

In [28]:
model2 = DecisionTreeClassifier(max_depth=5)
model2.fit(x_train, y_train)
y_pred = model2.predict(x_test)
print('train data set 정확도:' ,model2.score(x_train,y_train))
print('test data set 정확도',model2.score(x_test,y_test))
print('test data set 정확도', accuracy_score(y_test, y_pred))

train data set 정확도: 0.9399855386840202
test data set 정확도 0.9360780065005417
test data set 정확도 0.9360780065005417


- 모델 기여 변수 추출

In [29]:
importance = pd.DataFrame({'feature_names' : x.columns, 'coefficient': model2.feature_importances_})
importance

,feature_names,coefficient
0,count_1,0.371442
1,routine_flg,0.458823
2,period,0.169248
3,campaign_name_0_입회비반액할인,0.000151
4,campaign_name_1_입회비무료,0.000337
5,class_name_0_종일,0.000000
6,class_name_1_주간,0.000000
7,gender_F,0.000000
